# 0 - Imports python

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from langdetect import detect, DetectorFactory
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, TextVectorization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from scikeras.wrappers import KerasClassifier
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset


# 1 - Import des fichiers train.csv et test.csv

In [ ]:
train_path = "./assets/train.csv"
train = pd.read_csv(train_path)

test_path = "./assets/test.csv"
test = pd.read_csv(test_path)

In [ ]:
train.head()

In [ ]:
test.head()

# 2 - Traitement des données train

## 1 - Analyse des données

In [ ]:
label = "Labels"

print(train[label].unique())

In [ ]:
print("Valeurs Labels manquantes :", train[label].isna().sum())

In [ ]:
tag = "Text_Tag"
print(list(train[tag].unique()))

In [ ]:
print("Valeurs Text_Tag manquantes :", train[tag].isna().sum())

- Remplacement des null par "unknown"

In [ ]:
train[tag] = train[tag].fillna("unknown")

In [ ]:
text = "Text"
print("Valeurs Text manquantes :", train[text].isna().sum())

### Analyse du champs Text

In [ ]:
doublons = train[text].duplicated().sum()
print(doublons)

In [ ]:
len_text = "len_text"
word_text = "word_text"

train[len_text] = train[text].str.len()
train[word_text] = train[text].str.split().str.len()

print(train[[len_text, word_text]].describe())

In [ ]:
def repeated_letters(text):
    return bool(re.search(r"([a-zA-Z])\1{2,}", str(text)))

repeated_letters_column = "repeated_letters"

train[repeated_letters_column] = train[text].apply(repeated_letters)

print(train[repeated_letters_column].value_counts())

In [ ]:
train[train[repeated_letters_column] == True].head(8)

- Malgré la répétition des lettres, je constate que le texte reste "réel"

### Aide à la décision sur longueur du champs Text

In [ ]:
# Répartition du nombre de lettres
plt.figure(figsize=(10,6))
plt.hist(train[len_text], bins=30, color='skyblue', edgecolor='black')
plt.xlabel("Nombre de lettres")
plt.ylabel("Nombre de lignes")
plt.title("Histogramme du nombre de lettres dans len_text")
plt.grid(axis='y', alpha=0.75)
plt.show()


In [ ]:
# Moyenne et médiane
mean_len = train[len_text].mean()
median_len = train[len_text].median()
print("\nMoyenne :", mean_len)
print("Médiane :", median_len)

# Quartiles 25% et 75%
q25 = train[len_text].quantile(0.25)
q75 = train[len_text].quantile(0.75)
print("25% :", q25)
print("75% :", q75)

# Percentiles de 80% à 95% par pas de 5%
percentiles = train[len_text].quantile([0.8, 0.85, 0.9, 0.95])
print("\nPercentiles 80% à 95% :")
print(percentiles)

- Je prends la décision de rester en dessous de 200 caractères afin de limiter surtout les textes exotiques dépassant les 500 caratères.

### Décompte du nombre de tag par text

In [ ]:
train["count_tag"] = train[tag].str.count(",") + 1
train.head()

### Répartition suivant nombre de tags

In [ ]:
repartition = train["count_tag"].value_counts().sort_index()
repartition.plot(kind="bar")
plt.xlabel("Nombre de tags")
plt.ylabel("Nombre de lignes")
plt.title("Répartition du nombre de tags par entrée")
plt.show()

In [ ]:
mean = train["count_tag"].mean()

median = train["count_tag"].median()

q25 = train["count_tag"].quantile(0.25)
q75 = train["count_tag"].quantile(0.75)
q85 = train["count_tag"].quantile(0.85)
q95 = train["count_tag"].quantile(0.95)


print("Moyenne :", mean)
print("Médiane :", median)
print("25% :", q25)
print("75% :", q75)
print("85% :", q85)
print("95% :", q95)


- On pourrait limiter le nombre de tags à 3 sans perte de beaucoup d'informations dans nos données, à voir.


### Pré-nettoyage des données train

In [ ]:
train[text] = train[text].str.lower()
train[tag] = train[tag].str.lower()

In [ ]:
train[text] = train[text].str.strip()
train[tag] = train[tag].str.strip()

In [ ]:
train[tag] = train[tag].str.replace(","," ")

In [ ]:
all_tags = train[tag].str.split().explode()

In [ ]:
tag_counts = all_tags.value_counts()

In [ ]:
print("20 tags les plus fréquents :")
print(tag_counts.head(20))

In [ ]:
print("\n20 tags les moins fréquents :")
print(tag_counts.tail(20))

### Suppression des tags rares (<= 5 apparitions)

In [ ]:
seuil = 5

def remove_rare_tag(tag_string):
    tags = tag_string.split()
    tags_kept = [t for t in tags if tag_counts[t] > seuil]
    return " ".join(tags_kept)

train[tag] = train[tag].apply(remove_rare_tag)

print(train[train[tag].str.contains(r"\bhomeless\b", na=False)]) # test sur le tag homeless qui comptait 1 apparition

- Vérification de la non-création de tag "null"

In [ ]:
print(train[tag].isna().sum())

## 2 - Nettoyage des données train

### 0 - Roadmap

Suite à l'analyse précédente j'applique les actions suivantes :
- Limitation des tags à 3 (les plus représentés) sur la colonne "Text_Tag"
- Limitation à 200 caractères sur la colonne "Text"
- Concernant les labels :
    - Exclure les inconnues
    - Regrouper les labels
    - Ordonner logiquement les labels
    - Vérifier la représentation des classes
    - Réajuster si nécessaire

### 1 - Limitation des tags

In [ ]:
def top_tags(tag_string, n=3):
    tags = tag_string.split()
    tags_sorted = sorted(tags, key=lambda t: tag_counts[t], reverse=True)
    return " ".join(tags_sorted[:n])

train[tag] = train[tag].apply(top_tags)
train.head()

### 2 - Limitation du nombre de caractères de Text

#### Détection de langue

In [ ]:
DetectorFactory.seed = 0 

def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

train["lang"] = train[text].astype(str).apply(detect_language)


In [ ]:
lang = "lang"

train[lang].value_counts()

Suppression des autres langues que EN :

In [ ]:
train = train[train["lang"] == "en"].copy()

In [ ]:
nltk.download("stopwords")
nltk.download("wordnet")

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    words = text.split()
    words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]
    return " ".join(words)

train[tag] = train[tag].astype(str).apply(clean_text)
train[text] = train[text].astype(str).apply(clean_text)

In [ ]:
train[text] = train[text].str[:200]

### 3 - Traitements des Labels

#### Suppression des Unknown

In [ ]:
train = train[train["Labels"] != 4].copy()

In [ ]:
train[label].unique()

#### Regroupement de classes

In [ ]:
mapping_labels = {
    0: "False",
    1: "False",
    2: "Partially True",
    3: "Mostly True",
    5: "True"
}

train["Labels"] = train["Labels"].map(mapping_labels)

numeric_mapping = {
    "False": 0,
    "Partially True": 1,
    "Mostly True": 2,
    "True": 3
}

train["Labels"] = train["Labels"].map(numeric_mapping)

In [ ]:
train[label].unique()

In [ ]:
print(train["Labels"].value_counts())

In [ ]:
plt.figure(figsize=(8,5), facecolor="#2b2b2b")  # fond gris foncé

ax = train["Labels"].value_counts().sort_index().plot(
    kind="bar",
    color="skyblue",
    edgecolor="black"
)

ax.set_facecolor("#2b2b2b")         
ax.tick_params(colors="white")       
ax.yaxis.label.set_color("white")    
ax.xaxis.label.set_color("white")   
ax.title.set_color("white")         

plt.xlabel("Label")
plt.ylabel("Nombre d'entrées")
plt.title("Répartition des Labels après regroupement")
plt.grid(axis='y', alpha=0.75, color="gray") 
plt.show()

On constate une représentation plus importante des 0 (False) suite au regroupement des classes.

Je décide de conserver les 4 classes pour le moment, le modèle proposera des nuances dont je vérifierai la pertinence.

La proposition pour avoir une classification binaire sera de regrouper 0 et 1 et de regrouper 2 et 3 mais nous garderons un déséquilibre de l'ordre de 5k7 vs 3k6. L'avantage sera d'avoir une sortie binaire pour l'entraînement de mon modèle.

### 4 - Suppression des colonnes de travail

In [ ]:
train.head()

In [ ]:
train = train.drop(["len_text", "word_text", "repeated_letters", "count_tag", "lang"], axis=1)

In [ ]:
train.head()

# 3 - Préparation des jeux de données pour les modèles

## 1 - Création du jeu de validation

In [ ]:
text = "Text"
tag = "Text_Tag"
label = "Labels"

X = train[[text, tag]]
y = train[label]

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

print("Train:", X_train.shape, y_train.shape)
print("Validation:", X_val.shape, y_val.shape)
print("Test:", X_test.shape, y_test.shape)

# 4 - Entraînement du modèle Régression Logistique

## 0 - TF-IDF pour Text

#### Création des transformers

In [ ]:
text_transformer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))

tag_transformer = CountVectorizer(token_pattern=r"(?u)\b\w+\b")

#### Transformation des colonnes

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("text", text_transformer, "Text"),
        ("tags", tag_transformer, "Text_Tag")
    ]
)

#### Création du Pipeline

In [ ]:
model_pipeline = Pipeline([
    ("features", preprocessor),
    ("classifier", LogisticRegression(
        solver="lbfgs",
        max_iter=1000,
        class_weight="balanced",
        random_state=42
    ))
])

## 1 - Entraînement

In [ ]:
model_pipeline.fit(X_train, y_train)

## 2 - Évaluation

In [ ]:
y_pred = model_pipeline.predict(X_val)

print("=== Classification Report ===")
print(classification_report(y_val, y_pred, target_names=["False", "Partially True", "Mostly True", "True"]))

# --- Matrice de confusion ---
cm = confusion_matrix(y_val, y_pred)
labels = ["False", "Partially True", "Mostly True", "True"]

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels, cbar=False)
plt.xlabel("Predicted", fontsize=12)
plt.ylabel("Actual", fontsize=12)
plt.title("Confusion Matrix", fontsize=14)
plt.show()

In [ ]:
report = classification_report(y_val, y_pred, target_names=labels, output_dict=True)
df_report = pd.DataFrame(report).transpose()

# affichage stylisé
print(df_report)

# heatmap des scores
plt.figure(figsize=(8, 4))
sns.heatmap(df_report.iloc[:-1, :-1], annot=True, cmap="Greens", cbar=False, fmt=".2f")
plt.title("Classification Report (Precision / Recall / F1)")
plt.show()

# 5 - Entraînement du modèle Random Forest

## 1 - Pipeline

In [ ]:
model_pipeline2 = Pipeline([
    ("preprocess", preprocessor),
    ("clf", RandomForestClassifier(
        n_estimators=200,
        max_depth=None,
        random_state=42,
        n_jobs=-1
    ))
])

## 2 - Entraînement

In [ ]:
print(X_train.shape)
print(len(y_train))


In [ ]:
model_pipeline2.fit(X_train, y_train)

## 3 - Évaluation

In [ ]:
y_pred = model_pipeline2.predict(X_val)

# Rapport
print("=== Classification Report ===")
print(classification_report(y_val, y_pred, target_names=["False", "Partially True", "Mostly True", "True"]))

# --- Matrice de confusion graphique ---
cm = confusion_matrix(y_val, y_pred)
labels = ["False", "Partially True", "Mostly True", "True"]

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels, cbar=False)
plt.xlabel("Predicted", fontsize=12)
plt.ylabel("Actual", fontsize=12)
plt.title("Confusion Matrix - RandomForest", fontsize=14)
plt.show()

# 6 - Entraînement du modèle LSTM

## 1 - Préparation des données

In [ ]:
X_train_text = (X_train["Text"].astype(str) + " " + X_train["Text_Tag"].astype(str)).tolist()
X_val_text = (X_val["Text"].astype(str) + " " + X_val["Text_Tag"].astype(str)).tolist()
X_test_text = (X_test["Text"].astype(str) + " " + X_test["Text_Tag"].astype(str)).tolist()


In [ ]:
max_words = 5000 
max_len = 200       
embedding_dim = 64

vectorizer = TextVectorization(max_tokens=max_words, output_sequence_length=max_len)
vectorizer.adapt(X_train_text)


X_train_vec = vectorizer(X_train_text)
X_val_vec   = vectorizer(X_val_text)
X_test_vec  = vectorizer(X_test_text)




## 2 - Création du modèle

In [ ]:
model = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim),
    Bidirectional(LSTM(64, dropout=0.3, recurrent_dropout=0.3)),
    Dense(4, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## 3 - Entraînement

In [ ]:
history = model.fit(
    X_train_vec, np.array(y_train),
    validation_data=(X_val_vec, np.array(y_val)),
    epochs=6,
    batch_size=32,
)

## 4 - Évaluation

In [ ]:
# Prédictions
y_pred_prob = model.predict(X_test_vec)
y_pred = np.argmax(y_pred_prob, axis=1)

labels_names = ["False", "Partially True", "Mostly True", "True"]
print("=== Classification Report ===")
print(classification_report(y_val, y_pred, target_names=labels_names))

# Matrice de confusion graphique
cm = confusion_matrix(y_val, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", xticklabels=labels_names, yticklabels=labels_names, cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - LSTM")
plt.show()

- Modèle proche du hasard, fort overfit, plusieurs tests sur des modèles plus lourds et plus légers (8-64 neurones).

Conclusion ML : Dataset apparemment trop petit pour du Machine Learning de base. Difficulté à comprendre le sens des données ce qui donne lieu à de l'overfitting et de la prédiction proche du hasard influencée par la représentation des classes. Possibilité d'augmenter artificiellement l'accuracy en passant à 2 classes au lieu de 4 (25% -> 50%).

# 7 - Entraînement du modèle BERT

## 1 - Chargement du modèle (pré-entraîné, principe de BERT)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

## 2 - Transformation des jeux de données en DF

In [ ]:

train_df = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
val_df = pd.concat([X_val, y_val], axis=1).reset_index(drop=True)
test_df = pd.concat([X_test, y_test], axis=1).reset_index(drop=True)

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

## 3 - Création du modèle

In [ ]:
num_labels = train_df[label].nunique()

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels
)

## 4 - Entraînement

In [ ]:
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


## 5 - Évaluation

In [ ]:
results = trainer.evaluate(test_dataset)
print(results)